### OPERATIONS HAUT NIVEAU

In [2]:
import pandas as pd
import numpy as np
from time import time
rng = np.random.default_rng(seed=int(time()))
pd.__version__

'2.1.4'

In [3]:
url = "http://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
penguins_df = pd.read_csv(
    url,
    encoding="utf8"
)
penguins_df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


#### GROUP BY

In [16]:
# poids moyens des penguins par espèce ou sexe
# => slectionnner les champs utiles
# => les champs du groupby doivent être dans le df (comme en SQL)
gb = penguins_df[["body_mass_g","species","sex"]].groupby(by=["species","sex"])
# gb["body_mass_g"].mean()
grouped_df = gb["body_mass_g"].agg(["mean", "max"])
grouped_df

mean     max
species   sex                        
Adelie    FEMALE  3368.835616  3900.0
          MALE    4043.493151  4775.0
Chinstrap FEMALE  3527.205882  4150.0
          MALE    3938.970588  4800.0
Gentoo    FEMALE  4679.741379  5200.0
          MALE    5484.836066  6300.0

In [ ]:
# Notion d'index multiple: requêtage
grouped_df.loc["Adelie", "mean"]
grouped_df.loc[("Adelie", "FEMALE")]

In [17]:
# manipulation
grouped_df.swaplevel(1,0, axis=0)

,,mean,max
sex,species,,
FEMALE,Adelie,3368.835616,3900.0
MALE,Adelie,4043.493151,4775.0
FEMALE,Chinstrap,3527.205882,4150.0
MALE,Chinstrap,3938.970588,4800.0
FEMALE,Gentoo,4679.741379,5200.0
MALE,Gentoo,5484.836066,6300.0
